In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
import matplotlib.pyplot as plt



data = {
    'temperatura': Temperatura_IoT,
    'humedad': Humedad_IoT,
    'hora': hora,
    'dia': dia,
    'presencia': presencial,
    'radiacion': radiacion
}
df = pd.DataFrame(data)
df['consumo'] = (df['temperatura']*0.02 + df['humedad']*0.01 +
                 df['presencia']*0.3 + df['radiacion']*0.0005 +
                 np.random.normal(0, 0.05, n))

# Entrenamiento
X = df.drop(columns=['consumo'])
y = df['consumo']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = keras.Sequential([
    keras.layers.Dense(16, activation='relu', input_shape=(6,)),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
history = model.fit(X_train_scaled, y_train, epochs=50, validation_split=0.1)

# Graficamos la pérdida
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.legend()
plt.title("Pérdida durante el entrenamiento")
plt.show()


In [ ]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("modelo_consumo.tflite", "wb") as f:
    f.write(tflite_model)
